In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.drbart_evaluation import *

In [2]:
model_name = 'artificial_2'
log_name = 'test'
with open('../transformed_event_logs/artificial_start_end_2_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)
known_resources = ['1', 'Clark', 'Jane', 'Joe', 'Karsten']
known_activities = ['DIAGNOSIS', 'QUALITY_CONTROL', 'REPAIR']

In [3]:
test_event_log

,concept:name,lifecycle:transition_start,time:timestamp_start,org:resource,case:concept:name,id_start,lifecycle:transition_complete,time:timestamp_complete,id_complete,duration,duration_seconds,seconds_in_day,DIAGNOSIS,QUALITY_CONTROL,REPAIR,1,Clark,Jane,Joe,Karsten
0,DIAGNOSIS,START,2020-01-01 13:43:28.469233+00:00,Joe,0,1,COMPLETE,2020-01-01 14:59:41.608354+00:00,2,0 days 01:16:13.139121,4573,49408,1,0,0,0,0,0,1,0
1,REPAIR,START,2020-01-01 14:59:41.608354+00:00,Jane,0,4,COMPLETE,2020-01-04 18:38:11.639016+00:00,5,3 days 03:38:30.030662,272310,53981,1,0,1,0,0,1,1,0
3,DIAGNOSIS,START,2020-01-01 19:33:48.900814+00:00,1,1,10,COMPLETE,2020-01-01 20:34:39.406353+00:00,11,0 days 01:00:50.505539,3650,70428,1,0,0,1,0,0,0,0
4,REPAIR,START,2020-01-01 20:34:39.406353+00:00,1,1,13,COMPLETE,2020-01-02 09:05:17.659036+00:00,14,0 days 12:30:38.252683,45038,74079,1,0,1,2,0,0,0,0
6,DIAGNOSIS,START,2020-01-02 07:47:21.536525+00:00,Clark,2,19,COMPLETE,2020-01-02 08:45:47.275476+00:00,20,0 days 00:58:25.738951,3505,28041,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,QUALITY_CONTROL,START,2022-06-30 11:11:14.473595+00:00,Jane,890,8017,COMPLETE,2022-06-30 13:45:04.355487+00:00,8018,0 days 02:33:49.881892,9229,40274,1,1,1,0,0,3,0,0
2669,QUALITY_CONTROL,START,2022-06-30 12:13:46.487067+00:00,Joe,889,8008,COMPLETE,2022-06-30 19:40:59.136223+00:00,8009,0 days 07:27:12.649156,26832,44026,1,1,1,1,0,1,1,0
2673,DIAGNOSIS,START,2022-06-30 22:25:05.097535+00:00,Karsten,891,8020,COMPLETE,2022-06-30 23:15:20.606227+00:00,8021,0 days 00:50:15.508692,3015,80705,1,0,0,0,0,0,0,1
2674,REPAIR,START,2022-06-30 23:15:20.606227+00:00,Jane,891,8023,COMPLETE,2022-07-01 07:28:48.982352+00:00,8024,0 days 08:13:28.376125,29608,83720,1,0,1,0,0,1,0,1


In [4]:
N = 10000
n_processes = 10
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [5]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_Normal_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                       | 1/892 [08:10<121:17:00, 490.03s/it]

  0%|▏                                                                       | 3/892 [08:15<31:54:44, 129.23s/it]

  1%|▋                                                                         | 9/892 [08:21<7:46:02, 31.67s/it]

  2%|█▏                                                                       | 15/892 [08:42<4:08:43, 17.02s/it]

...

 99%|██████████████████████████████████████████████████████████████████████▉ | 879/892 [1:25:15<01:24,  6.50s/it]

 99%|███████████████████████████████████████████████████████████████████████▎| 883/892 [1:25:17<00:35,  3.99s/it]

 99%|███████████████████████████████████████████████████████████████████████▌| 887/892 [1:25:18<00:13,  2.65s/it]

100%|███████████████████████████████████████████████████████████████████████▊| 889/892 [1:25:24<00:08,  2.70s/it]

100%|████████████████████████████████████████████████████████████████████████| 892/892 [1:25:24<00:00,  5.74s/it]

In [6]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('-11.25201206507937534860318045')

In [7]:
np.mean(get_pscores(likelihoods_A))

np.float64(13145.750684272027)

In [8]:
drbart_model_A_R = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource/',
                     strict_parser=True)
evaluator_A_R = conduct_evaluation.ConductEvaluation(drbart_model_A_R, SampleOutcomes_DRBART_Normal_A_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R = evaluator_A_R.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [02:34<38:18:54, 154.81s/it]

  1%|▉                                                                        | 11/892 [02:52<2:52:32, 11.75s/it]

  2%|█▍                                                                       | 18/892 [02:53<1:29:33,  6.15s/it]

  3%|██                                                                         | 24/892 [02:55<57:29,  3.97s/it]

...

 98%|████████████████████████████████████████████████████████████████████████▍ | 873/892 [31:26<00:17,  1.10it/s]

 98%|████████████████████████████████████████████████████████████████████████▊ | 877/892 [31:26<00:08,  1.77it/s]

 99%|█████████████████████████████████████████████████████████████████████████▍| 885/892 [31:27<00:02,  3.04it/s]

 99%|█████████████████████████████████████████████████████████████████████████▌| 887/892 [31:29<00:02,  2.37it/s]

100%|██████████████████████████████████████████████████████████████████████████| 892/892 [31:29<00:00,  2.12s/it]

In [9]:
np.mean([v.ln() for v in likelihoods_A_R[0].values()])

Decimal('-11.11441252644552894451916162')

In [10]:
np.mean(get_pscores(likelihoods_A_R))

np.float64(10970.442952384437)

In [11]:
drbart_model_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/resource_seconds-in-day/',
                     strict_parser=True)
evaluator_R_S = conduct_evaluation.ConductEvaluation(drbart_model_R_S, SampleOutcomes_DRBART_Normal_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R_S = evaluator_R_S.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [03:23<50:24:05, 203.64s/it]

  0%|▏                                                                        | 2/892 [03:39<23:04:38, 93.35s/it]

  0%|▏                                                                        | 3/892 [03:49<13:39:48, 55.33s/it]

  1%|▋                                                                         | 9/892 [03:58<3:02:49, 12.42s/it]

...

 99%|██████████████████████████████████████████████████████████████████████▉ | 879/892 [1:23:57<01:27,  6.73s/it]

 99%|███████████████████████████████████████████████████████████████████████▏| 882/892 [1:24:15<01:04,  6.46s/it]

 99%|███████████████████████████████████████████████████████████████████████▌| 887/892 [1:24:22<00:20,  4.11s/it]

100%|███████████████████████████████████████████████████████████████████████▋| 888/892 [1:24:33<00:19,  4.97s/it]

100%|████████████████████████████████████████████████████████████████████████| 892/892 [1:24:33<00:00,  5.69s/it]

In [12]:
np.mean([v.ln() for v in likelihoods_R_S[0].values()])

Decimal('-11.60403755481397556367947650')

In [13]:
np.mean(get_pscores(likelihoods_R_S))

np.float64(20394.349061367157)

In [14]:
drbart_model_R = DRBART(parser_dir = '../../../models/'+model_name+'/resource/',
                     strict_parser=True)
evaluator_R = conduct_evaluation.ConductEvaluation(drbart_model_R, SampleOutcomes_DRBART_Normal_R,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_R = evaluator_R.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [03:57<58:40:20, 237.06s/it]

  0%|▏                                                                        | 2/892 [03:59<24:25:25, 98.79s/it]

  0%|▏                                                                        | 3/892 [04:11<14:36:08, 59.13s/it]

  0%|▎                                                                         | 4/892 [04:15<9:15:43, 37.55s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████▏| 882/892 [47:15<00:09,  1.02it/s]

 99%|█████████████████████████████████████████████████████████████████████████▍| 885/892 [47:16<00:05,  1.29it/s]

 99%|█████████████████████████████████████████████████████████████████████████▌| 887/892 [47:16<00:03,  1.54it/s]

100%|██████████████████████████████████████████████████████████████████████████| 892/892 [47:17<00:00,  2.55it/s]

100%|██████████████████████████████████████████████████████████████████████████| 892/892 [47:17<00:00,  3.18s/it]

In [15]:
np.mean([v.ln() for v in likelihoods_R[0].values()])

Decimal('-11.51530368846541280757874031')

In [16]:
np.mean(get_pscores(likelihoods_R))

np.float64(18127.009345817132)

In [17]:
drbart_model_A_R_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day/',
                     strict_parser=True)
evaluator_A_R_S = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S, SampleOutcomes_DRBART_Normal_A_R_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S = evaluator_A_R_S.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [05:54<87:45:39, 354.59s/it]

  0%|▏                                                                       | 2/892 [06:21<40:00:31, 161.83s/it]

  0%|▎                                                                        | 4/892 [06:53<17:21:33, 70.38s/it]

  1%|▋                                                                         | 9/892 [06:57<5:28:36, 22.33s/it]

...

 99%|███████████████████████████████████████████████████████████████████████▌| 887/892 [1:06:46<00:10,  2.04s/it]

100%|███████████████████████████████████████████████████████████████████████▋| 888/892 [1:06:52<00:11,  2.87s/it]

100%|███████████████████████████████████████████████████████████████████████▊| 889/892 [1:06:58<00:10,  3.50s/it]

100%|███████████████████████████████████████████████████████████████████████▊| 890/892 [1:07:02<00:07,  3.72s/it]

100%|████████████████████████████████████████████████████████████████████████| 892/892 [1:07:03<00:00,  4.51s/it]

In [18]:
np.mean([v.ln() for v in likelihoods_A_R_S[0].values()])

Decimal('-11.18146425945169256953372413')

In [19]:
np.mean(get_pscores(likelihoods_A_R_S))

np.float64(12567.945467881085)

In [20]:
drbart_model_A_R_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_R_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_AC, SampleOutcomes_DRBART_Normal_A_R_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_AC = evaluator_A_R_S_AC.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [02:06<31:23:26, 126.83s/it]

  0%|▏                                                                        | 2/892 [02:13<13:52:03, 56.09s/it]

  0%|▏                                                                         | 3/892 [02:21<8:26:16, 34.17s/it]

  1%|▍                                                                         | 5/892 [02:23<3:48:54, 15.48s/it]

...

 98%|████████████████████████████████████████████████████████████████████████▎ | 872/892 [28:01<00:15,  1.30it/s]

 98%|████████████████████████████████████████████████████████████████████████▋ | 876/892 [28:02<00:08,  1.84it/s]

 98%|████████████████████████████████████████████████████████████████████████▊ | 878/892 [28:08<00:15,  1.10s/it]

 99%|█████████████████████████████████████████████████████████████████████████▎| 884/892 [28:09<00:05,  1.50it/s]

100%|██████████████████████████████████████████████████████████████████████████| 892/892 [28:09<00:00,  1.89s/it]

In [21]:
np.mean([v.ln() for v in likelihoods_A_R_S_AC[0].values()])

Decimal('-11.50367400232841152278169693')

In [22]:
np.mean(get_pscores(likelihoods_A_R_S_AC))

np.float64(10518.447863273967)

In [23]:
drbart_model_A_R_S_RC_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_activity-count_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC_AC, SampleOutcomes_DRBART_Normal_A_R_S_RC_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                        'known_activities' : known_activities,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC_AC = evaluator_A_R_S_RC_AC.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [05:17<78:29:26, 317.13s/it]

  1%|▌                                                                         | 7/892 [05:20<8:16:08, 33.64s/it]

  1%|▉                                                                        | 11/892 [05:22<4:29:27, 18.35s/it]

  2%|█▍                                                                       | 17/892 [05:25<2:18:30,  9.50s/it]

...

 99%|█████████████████████████████████████████████████████████████████████████ | 881/892 [56:22<00:29,  2.64s/it]

100%|█████████████████████████████████████████████████████████████████████████▋| 888/892 [56:25<00:05,  1.49s/it]

100%|█████████████████████████████████████████████████████████████████████████▊| 889/892 [56:39<00:08,  2.74s/it]

100%|█████████████████████████████████████████████████████████████████████████▊| 890/892 [56:43<00:05,  2.83s/it]

100%|██████████████████████████████████████████████████████████████████████████| 892/892 [56:43<00:00,  3.82s/it]

In [24]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC_AC[0].values()])

Decimal('-11.05803755364135066690474654')

In [25]:
np.mean(get_pscores(likelihoods_A_R_S_RC_AC))

np.float64(10810.871368321828)

In [26]:
drbart_model_A_R_S_RC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_resource_seconds-in-day_resource-count/',
                     strict_parser=True)
evaluator_A_R_S_RC = conduct_evaluation.ConductEvaluation(drbart_model_A_R_S_RC, SampleOutcomes_DRBART_Normal_A_R_S_RC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                        'known_resources' : known_resources,
                                                    },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_R_S_RC = evaluator_A_R_S_RC.sample_cases(False, True)

  0%|                                                                                    | 0/892 [00:00<?, ?it/s]

  0%|                                                                        | 1/892 [05:06<75:48:33, 306.30s/it]

  1%|▍                                                                         | 6/892 [05:14<9:37:29, 39.11s/it]

  1%|▋                                                                         | 9/892 [05:24<5:49:01, 23.72s/it]

  1%|▉                                                                        | 12/892 [05:33<3:53:01, 15.89s/it]

...

 98%|████████████████████████████████████████████████████████████████████████▊ | 877/892 [48:54<00:13,  1.13it/s]

 99%|█████████████████████████████████████████████████████████████████████████▏| 882/892 [48:55<00:05,  1.90it/s]

 99%|█████████████████████████████████████████████████████████████████████████▍| 885/892 [48:57<00:03,  1.93it/s]

100%|█████████████████████████████████████████████████████████████████████████▉| 891/892 [48:57<00:00,  2.98it/s]

100%|██████████████████████████████████████████████████████████████████████████| 892/892 [48:57<00:00,  3.29s/it]

In [27]:
np.mean([v.ln() for v in likelihoods_A_R_S_RC[0].values()])

Decimal('-11.17145522366979133157422506')

In [28]:
np.mean(get_pscores(likelihoods_A_R_S_RC))

np.float64(12977.021086436893)

In [29]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_R' : likelihoods_A_R,
    'drbart_model_R' : likelihoods_R,
    'drbart_model_R_S' : likelihoods_R_S,
    'drbart_model_A_R_S' : likelihoods_A_R_S,
    'drbart_model_A_R_S_AC' : likelihoods_A_R_S_AC,
    'drbart_model_A_R_S_RC' : likelihoods_A_R_S_RC,
    'drbart_model_A_R_S_RC_AC' : likelihoods_A_R_S_RC_AC
}

with open('./'+model_name+'_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)